In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import root_mean_squared_error, r2_score
import pickle
import mlflow


%matplotlib inline

In [2]:
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
mlflow.set_experiment('duration-prediction')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1729865503894, experiment_id='1', last_update_time=1729865503894, lifecycle_stage='active', name='duration-prediction', tags={}>

In [3]:
df = pd.read_parquet('../data/yellow_tripdata_2024-07.parquet')
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-07-01 00:34:56,2024-07-01 00:46:49,1.0,3.20,1.0,N,140,79,1,15.6,3.50,0.5,3.50,0.00,1.0,24.10,2.5,0.00
1,2,2024-06-30 23:48:58,2024-07-01 00:28:04,1.0,19.48,2.0,N,132,113,2,70.0,0.00,0.5,0.00,0.00,1.0,75.75,2.5,1.75
2,2,2024-07-01 00:23:18,2024-07-01 00:29:51,1.0,1.18,1.0,N,237,145,1,8.6,1.00,0.5,2.72,0.00,1.0,16.32,2.5,0.00
3,1,2024-07-01 00:10:33,2024-07-01 00:27:31,0.0,9.10,1.0,N,138,164,1,36.6,10.25,0.5,12.05,0.00,1.0,60.40,2.5,1.75
4,1,2024-07-01 00:07:55,2024-07-01 00:34:34,1.0,17.70,2.0,N,132,263,1,70.0,1.75,0.5,10.00,6.94,1.0,90.19,0.0,1.75


In [4]:
del df

Read Data

In [5]:
categories = ['PULocationID','DOLocationID']
numerics =  [] #['trip_distance','passenger_count','fare_amount','tip_amount','congestion_surcharge','Airport_fee','tpep_pickup_datetime']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    df['duration']= df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].map(lambda x: x.total_seconds()/60)
    
    df = df[(df['duration'] >= 0) & (df['duration'] <= 60)  ]
    df[categories] = df[categories].astype(object)

    df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].astype(int)

    prepped = df[categories + numerics + ['duration']].dropna()
    return prepped

In [6]:
train_data = read_data('../data/yellow_tripdata_2024-07.parquet')

In [7]:
train_data['duration'].describe(percentiles = [.5,.95,.99])

count    3.022755e+06
mean     1.549464e+01
std      1.111104e+01
min      0.000000e+00
50%      1.258333e+01
95%      3.926667e+01
99%      5.356667e+01
max      6.000000e+01
Name: duration, dtype: float64

In [8]:
train_data.head()

,PULocationID,DOLocationID,duration
0,140,79,11.883333
1,132,113,39.100000
2,237,145,6.550000
3,138,164,16.966667
4,132,263,26.650000


Training pipeline


In [9]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_data[categories + numerics].to_dict(orient='records'))

In [10]:
dv.feature_names_

['DOLocationID', 'PULocationID']

In [11]:
target = 'duration'
y_train = train_data[target].values
y_train


array([11.88333333, 39.1       ,  6.55      , ..., 22.        ,
       18.1       , 18.41666667])

In [12]:
test_data = read_data('../data/yellow_tripdata_2024-08.parquet')
x_test_dict = test_data[categories + numerics].to_dict(orient='records')
x_test = dv.transform(x_test_dict)
y_test = test_data[target].values

Linear Regression

In [13]:
%pwd

'/home/codespace/mlops-zoomcamp/chapter2'

In [14]:
with mlflow.start_run():
    mlflow.set_tag('developer',value='Derek')
    mlflow.set_tag('model',value='linear_regression')

    
    lm = LinearRegression()
    lm.fit(x_train, y_train)
    preds = lm.predict(x_test)
    rmse= root_mean_squared_error(y_test,preds)
    r2 = r2_score(y_test,preds)
    print(f'The model has an average error of {rmse:.2f} minutes,while explaining {r2:.2f} of the variance')
    mlflow.log_params(params=lm.get_params())
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r2',r2)
    mlflow.log_artifact('../models/intro_lm.bin','model')



The model has an average error of 11.08 minutes,while explaining 0.02 of the variance


2024/10/28 15:50:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run learned-skunk-377 at: http://127.0.0.1:5000/#/experiments/1/runs/05a14157c63b453fa14168426b18573b.
2024/10/28 15:50:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


In [15]:
resids = pd.DataFrame({'actual':y_test,'predicted':preds})
resids['residual'] = resids['actual'] - resids['predicted']
resids['abs_resid'] = resids['residual'].abs()
resids.sort_values('abs_resid',ascending=False)

,actual,predicted,residual,abs_resid
2434206,59.933333,12.168918,47.764415,47.764415
1533981,59.916667,12.168918,47.747748,47.747748
335859,60.000000,12.267402,47.732598,47.732598
861565,59.766667,12.168918,47.597748,47.597748
1165082,59.683333,12.193165,47.490169,47.490169
...,...,...,...,...
2158445,14.900000,14.900020,-0.000020,0.000020
2158444,14.900000,14.900020,-0.000020,0.000020
1266413,17.800000,17.799982,0.000018,0.000018
2783444,17.433333,17.433338,-0.000004,0.000004


In [16]:
with open('../models/intro_lm.bin','wb') as f_out:
    pickle.dump((dv,lm),f_out)

Ridge Regression

In [17]:
with mlflow.start_run():
    mlflow.set_tag('developer',value='Derek')
    mlflow.set_tag('model',value='ridge_regression')
    alpha = 0.03
    mlflow.log_param('alpha',alpha)

    ridge = Ridge(alpha=alpha)
    ridge.fit(x_train,y_train)
    preds = ridge.predict(x_test)
    rmse= root_mean_squared_error(y_test,preds)
    r2 = r2_score(y_test,preds)
    print(f'The model has an average error of {rmse:.2f} minutes,while explaining {r2:.2f} of the variance')
    mlflow.log_params(params=ridge.get_params())
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r2',r2)
    mlflow.log_artifact('../models/intro_ridge.bin','model')

2024/10/28 15:50:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run efficient-tern-750 at: http://127.0.0.1:5000/#/experiments/1/runs/9cf13a3510f34780b0c51007ca3d022e.
2024/10/28 15:50:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


The model has an average error of 11.08 minutes,while explaining 0.02 of the variance


In [18]:
with open('../models/intro_ridge.bin','wb') as f_out:
    pickle.dump((dv,ridge),f_out)

Lasso Regression

In [19]:
with mlflow.start_run():    
    mlflow.set_tag('developer',value='Derek')
    mlflow.set_tag('model',value='lasso_regression')
    alpha = 0.03
    mlflow.log_param('alpha',alpha)

    lasso = Lasso(alpha=alpha)
    lasso.fit(x_train,y_train)
    preds = lasso.predict(x_test)
    rmse= root_mean_squared_error(y_test,preds)
    r2 = r2_score(y_test,preds)
    print(f'The model has an average error of {rmse:.2f} minutes,while explaining {r2:.2f} of the variance')
    mlflow.log_params(params=lasso.get_params())
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r2',r2)
    mlflow.log_artifact('../models/intro_lasso.bin','model')

2024/10/28 15:50:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-carp-263 at: http://127.0.0.1:5000/#/experiments/1/runs/90ffd292c7884eb4ac8f2c5383958b14.
2024/10/28 15:50:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


The model has an average error of 11.08 minutes,while explaining 0.02 of the variance


In [20]:
with open('../models/intro_lasso.bin','wb') as f_out:
    pickle.dump((dv,lasso),f_out)

# Hyperparameter tuning with mlflow and hyperopt

In [21]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

In [22]:
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_test, label=y_test)

objective function for xgboost

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)

        booster = xgb.train(params=params, 
                            dtrain=train,
                            num_boost_round=1000,
                            evals=[(valid, 'validation')],
                            early_stopping_rounds=50)
        preds = booster.predict(valid)
        rmse = np.sqrt(root_mean_squared_error(y_test, preds))
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search space
area that hyperopt will search for the best parameters
all are xgboost parameters
read more here: https://hyperopt.github.io/hyperopt/getting-started/search_spaces/#parameter-expressions
and here: https://xgboost.readthedocs.io/en/latest/parameter.html

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate': hp.loguniform('learning_rate',-3,0),
    'reg_alpha': hp.loguniform('reg_alpha',-5, -1),
    'reg_lambda': hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight': scope.int(hp.loguniform('min_child_weight',-1,3)),
    'objective': 'reg:squarederror',
    'seed': 192
}
#each one is a range that hyperopt will use

optimisation using fmin
minimises the metric provided using the search space

In [25]:
best_result = fmin(objective,
                   space=search_space,
                   algo=tpe.suggest,
                   max_evals=10,
                   trials=Trials())


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:7.67575                           
[1]	validation-rmse:6.85029                           
[2]	validation-rmse:6.68857                           
[3]	validation-rmse:6.65806                           
[4]	validation-rmse:6.65172                           
[5]	validation-rmse:6.65004                           
[6]	validation-rmse:6.64926                           
[7]	validation-rmse:6.64901                           
[8]	validation-rmse:6.64880                           
[9]	validation-rmse:6.64860                           
[10]	validation-rmse:6.64856                          
[11]	validation-rmse:6.64848                          
[12]	validation-rmse:6.64846                          
[13]	validation-rmse:6.64847                          
[14]	validation-rmse:6.64843                          
[15]	validation-rmse:6.64839                          
[16]	validation-rmse:6.64838                          
[17]	validation-rmse:6.64836                          
[18]	valid

2024/10/28 15:51:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run classy-flea-640 at: http://127.0.0.1:5000/#/experiments/1/runs/7aa1223cd0de4c2081d417622fb5c2f0.

2024/10/28 15:51:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



[0]	validation-rmse:9.99029                                                    
[1]	validation-rmse:9.07122                                                    
[2]	validation-rmse:8.38424                                                    
[3]	validation-rmse:7.87948                                                    
[4]	validation-rmse:7.51430                                                    
[5]	validation-rmse:7.25359                                                    
[6]	validation-rmse:7.06944                                                    
[7]	validation-rmse:6.94041                                                    
[8]	validation-rmse:6.85055                                                    
[9]	validation-rmse:6.78822                                                    
[10]	validation-rmse:6.74512                                                   
[11]	validation-rmse:6.71535                                                   
[12]	validation-rmse:6.69482            

2024/10/28 15:52:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run big-quail-450 at: http://127.0.0.1:5000/#/experiments/1/runs/a4dd402a92d84c90b17225ad40d5bd08.

2024/10/28 15:52:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



...

[0]	validation-rmse:9.41495                                                     
[1]	validation-rmse:8.27887                                                    
[2]	validation-rmse:7.58574                                                    
[3]	validation-rmse:7.17808                                                    
[4]	validation-rmse:6.94467                                                    
[5]	validation-rmse:6.81331                                                    
[6]	validation-rmse:6.74014                                                    
[7]	validation-rmse:6.69954                                                    
[8]	validation-rmse:6.67706                                                    
[9]	validation-rmse:6.66461                                                    
[10]	validation-rmse:6.65768                                                   
[11]	validation-rmse:6.65382                                                   
[12]	validation-rmse:6.65165           

2024/10/28 16:01:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run thoughtful-crab-956 at: http://127.0.0.1:5000/#/experiments/1/runs/71c1a788771444d6af944ed301d22f00.

2024/10/28 16:01:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



[0]	validation-rmse:10.75662                                                   
[1]	validation-rmse:10.35328                                                   
[2]	validation-rmse:9.98597                                                    
[3]	validation-rmse:9.65170                                                    
[4]	validation-rmse:9.34846                                                    
[5]	validation-rmse:9.07341                                                    
[6]	validation-rmse:8.82502                                                    
[7]	validation-rmse:8.60079                                                    
[8]	validation-rmse:8.39828                                                    
[9]	validation-rmse:8.21562                                                    
[10]	validation-rmse:8.05151                                                   
[11]	validation-rmse:7.90472                                                   
[12]	validation-rmse:7.77314            

2024/10/28 16:04:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run youthful-pig-53 at: http://127.0.0.1:5000/#/experiments/1/runs/c61feffd27d8425e9f2e634d0f858512.

2024/10/28 16:04:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



100%|██████████| 10/10 [13:58<00:00, 83.85s/trial, best loss: 2.578424661145273]


In [26]:
best_result

{'learning_rate': np.float64(0.7642238261917301),
 'max_depth': np.float64(89.0),
 'min_child_weight': np.float64(12.978126207985504),
 'reg_alpha': np.float64(0.046390865549832794),
 'reg_lambda': np.float64(0.03898891619999199)}

In [28]:
best_result['max_depth'] = int(best_result['max_depth'])

best model

In [29]:
with mlflow.start_run():
    mlflow.set_tag('model', 'xgboost')
    mlflow.log_params(best_result)

    booster = xgb.train(params=best_result, 
                        dtrain=train,
                    num_boost_round=1000,
                    evals=[(valid, 'validation')],
                    early_stopping_rounds=50)


[0]	validation-rmse:6.99003
[1]	validation-rmse:6.67306
[2]	validation-rmse:6.65297
[3]	validation-rmse:6.65064
[4]	validation-rmse:6.64966
[5]	validation-rmse:6.64923
[6]	validation-rmse:6.64896
[7]	validation-rmse:6.64885
[8]	validation-rmse:6.64866
[9]	validation-rmse:6.64857
[10]	validation-rmse:6.64842
[11]	validation-rmse:6.64833
[12]	validation-rmse:6.64825
[13]	validation-rmse:6.64830
[14]	validation-rmse:6.64825
[15]	validation-rmse:6.64818
[16]	validation-rmse:6.64812
[17]	validation-rmse:6.64814
[18]	validation-rmse:6.64813
[19]	validation-rmse:6.64810
[20]	validation-rmse:6.64809
[21]	validation-rmse:6.64812
[22]	validation-rmse:6.64809
[23]	validation-rmse:6.64808
[24]	validation-rmse:6.64809
[25]	validation-rmse:6.64806
[26]	validation-rmse:6.64807
[27]	validation-rmse:6.64806
[28]	validation-rmse:6.64810
[29]	validation-rmse:6.64808
[30]	validation-rmse:6.64807
[31]	validation-rmse:6.64805
[32]	validation-rmse:6.64810
[33]	validation-rmse:6.64807
[34]	validation-rmse:6.6

2024/10/28 16:06:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run bald-worm-469 at: http://127.0.0.1:5000/#/experiments/1/runs/922cbeea08dd404ebf31e87789ed3470.
2024/10/28 16:06:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


auto-logging
read more here: https://mlflow.org/docs/latest/tracking.html#automatic-logging

In [30]:
best_result['max_depth'] = int(best_result['max_depth'])


In [31]:
mlflow.xgboost.autolog(disable=True)

In [32]:
mlflow.xgboost.autolog()
booster = xgb.train(params=best_result, 
                    dtrain=train,
                    num_boost_round=1000,
                    evals=[(valid, 'validation')],
                    early_stopping_rounds=50)

2024/10/28 16:07:01 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.1, but the installed version is 2.1.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2024/10/28 16:07:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e47a325749af45219c77c78cfbcee18e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:6.99003
[1]	validation-rmse:6.67306
[2]	validation-rmse:6.65297
[3]	validation-rmse:6.65064
[4]	validation-rmse:6.64966
[5]	validation-rmse:6.64923
[6]	validation-rmse:6.64896
[7]	validation-rmse:6.64885
[8]	validation-rmse:6.64866
[9]	validation-rmse:6.64857
[10]	validation-rmse:6.64842
[11]	validation-rmse:6.64833
[12]	validation-rmse:6.64825
[13]	validation-rmse:6.64830
[14]	validation-rmse:6.64825
[15]	validation-rmse:6.64818
[16]	validation-rmse:6.64812
[17]	validation-rmse:6.64814
[18]	validation-rmse:6.64813
[19]	validation-rmse:6.64810
[20]	validation-rmse:6.64809
[21]	validation-rmse:6.64812
[22]	validation-rmse:6.64809
[23]	validation-rmse:6.64808
[24]	validation-rmse:6.64809
[25]	validation-rmse:6.64806
[26]	validation-rmse:6.64807
[27]	validation-rmse:6.64806
[28]	validation-rmse:6.64810
[29]	validation-rmse:6.64808
[30]	validation-rmse:6.64807
[31]	validation-rmse:6.64805
[32]	validation-rmse:6.64810
[33]	validation-rmse:6.64807
[34]	validation-rmse:6.6

2024/10/28 16:07:50 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/10/28 16:07:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:07:50] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/10/28 16:07:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 16:07:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run rare-koi-987 at: http://127.0.0.1:5000/#/experiments/1/runs/e47a325749af45219c77c78cfbcee18e.
2024/10/28 16:07:54 INFO mlflow.tracking._tracking_s

In [33]:
mlflow.xgboost.autolog(disable=True)

In [38]:
with mlflow.start_run():
   booster = xgb.train(params=best_result, 
                        dtrain=train,
                        num_boost_round=1000,
                        evals=[(valid, 'validation')],
                        early_stopping_rounds=50)
   mlflow.log_params(params=best_result)
   preds = booster.predict(valid)
   rmse = np.sqrt(root_mean_squared_error(y_test, preds))
   mlflow.log_metric('rmse', rmse)
   mlflow.set_tag('model', 'xgboost')
   mlflow.xgboost.log_model(booster,artifact_path='models')
   with open('../models/preprocessor.b','wb') as f_out:
      pickle.dump(dv,f_out)
   mlflow.log_artifact('../models/preprocessor.b', artifact_path='preprocessor')
   
    

[0]	validation-rmse:6.99003
[1]	validation-rmse:6.67306
[2]	validation-rmse:6.65297
[3]	validation-rmse:6.65064
[4]	validation-rmse:6.64966
[5]	validation-rmse:6.64923
[6]	validation-rmse:6.64896
[7]	validation-rmse:6.64885
[8]	validation-rmse:6.64866
[9]	validation-rmse:6.64857
[10]	validation-rmse:6.64842
[11]	validation-rmse:6.64833
[12]	validation-rmse:6.64825
[13]	validation-rmse:6.64830
[14]	validation-rmse:6.64825
[15]	validation-rmse:6.64818
[16]	validation-rmse:6.64812
[17]	validation-rmse:6.64814
[18]	validation-rmse:6.64813
[19]	validation-rmse:6.64810
[20]	validation-rmse:6.64809
[21]	validation-rmse:6.64812
[22]	validation-rmse:6.64809
[23]	validation-rmse:6.64808
[24]	validation-rmse:6.64809
[25]	validation-rmse:6.64806
[26]	validation-rmse:6.64807
[27]	validation-rmse:6.64806
[28]	validation-rmse:6.64810
[29]	validation-rmse:6.64808
[30]	validation-rmse:6.64807
[31]	validation-rmse:6.64805
[32]	validation-rmse:6.64810
[33]	validation-rmse:6.64807
[34]	validation-rmse:6.6

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:19:21] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/10/28 16:19:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 16:19:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run shivering-stag-340 at: http://127.0.0.1:5000/#/experiments/1/runs/8f80585992ee47cfb3a3a7f19ccc4ab4.
2024/10/28 16:19:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


In [39]:

logged_model = 'runs:/730ce618f1d5499590a3008b364a0c14/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 730ce618f1d5499590a3008b364a0c14

In [40]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model
#load as a xgboost model

In [ ]:
preds = xgboost_model.predict(valid)